In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
df = pd.read_csv("lightcast_job_postings.csv")
print("Available columns in dataset:", df.columns.tolist())

columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_5"
]

df.drop(columns=columns_to_drop, inplace=True)
print("Dropped unnecessary columns.")
print(df.columns)

# handle missing value
print("Missing values before cleaning:")
print(df.isnull().sum())



import missingno as msno
import matplotlib.pyplot as plt

# Check column names
df.columns = df.columns.str.upper().str.strip()  # Normalize column names
print(df.columns)  # Debugging step

# Visualize missing data
plt.figure(figsize=(1, 0.5))
msno.heatmap(df)
plt.title("Missing Values Heatmap")
plt.show()

# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)

# Check if "SALARY" exists before filling missing values
if "SALARY" in df.columns:
    df["SALARY"].fillna(df["SALARY"].median(), inplace=True)
else:
    print("⚠️ Warning: 'SALARY' column not found in dataframe!")

# Check if "INDUSTRY" exists before filling missing values
if "INDUSTRY" in df.columns:
    df["INDUSTRY"].fillna("Unknown", inplace=True)
else:
    print("⚠️ Warning: 'INDUSTRY' column not found in dataframe!")

print("✅ Missing value handling complete.")

# delete duplicates
df = df.drop_duplicates(subset=["TITLE", "COMPANY", "LOCATION", "POSTED"])
print("Duplicates removed.")


: 

In [2]:
if "SALARY" in df.columns:
    df["SALARY"].fillna(df["SALARY"].median(), inplace=True)
else:
    print("⚠️ Warning: 'SALARY' column not found in dataframe!")

⚠️ Warning: 'SALARY' column not found in dataframe!


In [3]:
# identifying data analyst jobs by keyword searching
keywords = ['Data Analyst', 'Business Analyst', 'Data Engineering', 'Deep Learning',
            'Data Science', 'Data Analysis','Data Analytics',  'Market Research Analyst' 
            'LLM', 'Language Model', 'NLP', 'Natural Language Processing',
            'Computer Vision', 'Business Intelligence Analyst', 'Quantitative Analyst', 'Operations Analyst']

match = lambda col: df[col].str.contains('|'.join(keywords), case=False, na=False)

df['DATA_ANALYST_JOB'] = match('TITLE_NAME') \
             | match('SKILLS_NAME') \
             | match('SPECIALIZED_SKILLS_NAME') 
df['DATA_ANALYST_JOB'].value_counts()



DATA_ANALYST_JOB
False    37052
True     32148
Name: count, dtype: int64

In [4]:
df['DATA_ANALYST_JOB']

0        False
1        False
2         True
3        False
4        False
         ...  
72471     True
72472     True
72473     True
72474     True
72475    False
Name: DATA_ANALYST_JOB, Length: 69200, dtype: bool

In [5]:
import plotly.express as px

# Group data
df_grouped = df.groupby(['DATA_ANALYST_JOB', 'NAICS2_NAME']).size().reset_index(name='Job_Count')

# Define custom colors for 'True' and 'False' categories
color_map = {
    False: "#d62728",   # Red
    True: "#3BB143"     # Green (Fixed extra space issue)
}

# Create the figure
fig = px.bar(df_grouped, 
             x='NAICS2_NAME', 
             y='Job_Count',
             color='DATA_ANALYST_JOB',
             title="Data Analytics & Business Analytics Job Trends",
             labels={'NAICS2_NAME': 'Industry', 'Job_Count': 'Number of Jobs'},
             barmode='group',
             color_discrete_map=color_map  # Custom colors
            )

# **Fix sizing issues**
fig.update_layout(
    autosize=True,  # Let it adapt dynamically
    width=1200,     # Set wider width
    height=700,     # Set taller height
    margin=dict(l=60, r=60, t=50, b=100),  # Avoid excessive padding
    font=dict(size=14),  # Increase text size
    xaxis=dict(
        tickangle=-45,  # Rotate x-axis labels
        title_font=dict(size=16),  # Bigger title
        tickfont=dict(size=14)  # Bigger tick labels
    ),
    yaxis=dict(
        title="Number of Jobs", 
        range=[0, df_grouped['Job_Count'].max() * 1.2],  # Extend Y-axis range
        title_font=dict(size=16),  # Bigger title
        tickfont=dict(size=14)  # Bigger tick labels
    )
)

# Show the plot
fig.show()

# Save as an HTML file for Quarto embedding
fig.write_html("figures/plot1_jobtrends.html")


In [8]:
import plotly.express as px

# Identify the top 10 industries with the most data analyst jobs
top_industries = df_grouped.groupby("NAICS2_NAME")["Job_Count"].sum().nlargest(10).index

# Filter the dataset for top industries
df_top_industries = df_grouped[df_grouped["NAICS2_NAME"].isin(top_industries)]

# Create the bar chart
fig = px.bar(df_top_industries, 
             x="DATA_ANALYST_JOB", 
             y="Job_Count", 
             color="NAICS2_NAME",
             title="Top 10 Industries Hiring Data Analysts",
             labels={'DATA_ANALYST_JOB': 'Job Title', 'Job_Count': 'Number of Jobs'},
             barmode='group')

# **Fix sizing & layout**
fig.update_layout(
    autosize=True,
    width=1100,  # Adjust width to fit layout
    height=600,  # Increase height
    margin=dict(l=50, r=50, t=50, b=50),  # Reduce margins
    font=dict(size=14),  # Improve font readability
    xaxis=dict(tickangle=-45),  # Rotate x-axis labels for better fit
)
fig.show()
# Save the updated plot as an HTML file
fig.write_html("figures/plot2industries.html")


In [8]:
# Define target industries
target_industries = ["Finance and Insurance", "Health Care and Social Assistance"]

# Filter the DataFrame
df_skills = df[df["NAICS2_NAME"].isin(target_industries)]


In [11]:
import plotly.express as px

# Define target industries
target_industries = ["Finance and Insurance", "Health Care and Social Assistance"]

# Filter the original DataFrame (df) instead of df_grouped
df_remote = df[df["NAICS2_NAME"].isin(target_industries)]

# Count occurrences of each remote type in these industries
df_remote_grouped = df_remote.groupby("REMOTE_TYPE_NAME").size().reset_index(name="Count")

# Create a pie chart
fig = px.pie(df_remote_grouped, 
             names="REMOTE_TYPE_NAME", 
             values="Count", 
             title="Remote vs. On-Site Data Analyst Jobs in Finance & Healthcare",
             color="REMOTE_TYPE_NAME", 
             color_discrete_map={"Remote": "#1f77b4", "On-Site": "#d62728", "Hybrid": "#2ca02c"})

fig.show()
fig.write_html("plot3pie.html")
